In [1]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()

 Activating environment at `C:\Users\sbirk\Documents\sciebo\Programmierung\ATIS3_2020\Project.toml`


In [2]:
using JuMP, GLPK, LinearAlgebra, Plots, StatsPlots, CPLEX, Statistics

In [3]:
hexagons = [1, 2, 3, 4, 5]
zeitfenster = 8 # h # timeslots = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
charger_cost = 2000
# charger_types = [3, 11, 22]
# charger_cost = [1000, 2500, 4000]
# arrival_times = [17, 18, 19, 20]
# arrival_probability = [0.2, 0.3, 0.3, 0.2]
# departure_times = [6, 7, 8, 9]
# departure_probability = [0.2, 0.3, 0.3, 0.2]
demand = [30 20 10 5;
        30 20 10 5;
        30 20 10 5;
        30 20 10 5;
        30 20 10 5]
# soc = [30, 20, 10, 1]
demand_probability = [0.2, 0.3, 0.4, 0.1]
opportunity_cost = 100

4-element Array{Float64,1}:
 0.2
 0.3
 0.4
 0.1

In [ ]:
function charger_siting(charger_cost, opportunity_cost)
    Ω = length(demand_probability)
    Γ = Array{int}(1:length(hexagons))
    m = Model(GLPK.Optimizer)
    
    @variable(m, x_p[i = Γ] >= 0) # charging power in each hexagon
    @variable(m, y[i = Γ,1:Ω] >= 0) # supplied energy in each hexagon
    @variable(m, z[i = Γ,1:Ω] >= 0) # demand not covered
    
    
    @constraint(m, [ω in 1:Ω], y + z == demand[])
    @objective(m, Min, 
        sum(charger_cost .* x_p[i],for i in Γ)
        + sum(opportunity_cost .* z[]))
    